# Glue Spark

- RDD: Datasets which storage data in distributed format
- Transformations: Datasets iterations
- Actions: Run the transformations on RDD
- Output data: Result of transformations

Spark runs lazy, so transformations are not immediately executed, they stay in a logical plan to be executed when the actions are called.

Actions examples:

- `take()`
- `collect()`
- `show()`
- `save()`
- `count()`

## Types of transformations

### Narrow

Act only on one partition of the output data (Map, Filter)

- Does not requires to share data within another workers
- Are independent of other partitions
- Are computationally efficient

### Wide

Can work on several partitions of output data (Join, GroupBy)

## Running locally

1. Docker Compose file:

```yaml
services:
  glue:
    image: amazon/aws-glue-libs:glue_libs_4.0.0_image_01
    container_name: glue-jupyter
    ports:
      - "8889:8888"
      - "4040:4040"
    volumes:
      - .:/home/glue_user/workspace/jupyter_workspace
      - ~/.aws:/home/glue_user/.aws:ro
    working_dir: /home/glue_user/workspace
    environment:
      AWS_PROFILE: default
      DISABLE_SSL: true
    command: >
      /home/glue_user/jupyter/jupyter_start.sh
```

2. Run image:

```sh
docker compose run --rm glue
```

3. Connect to jupyter server within vscode:

- Select kernel
- Existing jupyter server
- https://127.0.1:8889/lab


Import packages


In [ ]:
from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create session


In [ ]:
spark = SparkSession.builder.appName("example").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Create a dataframe


In [ ]:
data = [
    ("John", "Sales", 3000),
    ("Bryan", "Sales", 4200),
    ("Selena", "Sales", 4600),
    ("Alice", "Sales", 3500),
    ("Mark", "Sales", 3900),
    ("Sophia", "Sales", 4100),
    ("Daniel", "Sales", 3800),
    ("Emma", "Sales", 4400),
    ("Lucas", "Sales", 3600),
    ("Olivia", "Sales", 4700),
]
df = spark.createDataFrame(data, ["name", "department", "salary"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Visualize data


In [ ]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+------+
|  name|department|salary|
+------+----------+------+
|  John|     Sales|  3000|
| Bryan|     Sales|  4200|
|Selena|     Sales|  4600|
| Alice|     Sales|  3500|
|  Mark|     Sales|  3900|
|Sophia|     Sales|  4100|
|Daniel|     Sales|  3800|
|  Emma|     Sales|  4400|
| Lucas|     Sales|  3600|
|Olivia|     Sales|  4700|
+------+----------+------+

Filtering dataset (narrow)


In [ ]:
df_filtered = df.filter(df["salary"] > 4000)
df_filtered.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+------+
|  name|department|salary|
+------+----------+------+
| Bryan|     Sales|  4200|
|Selena|     Sales|  4600|
|Sophia|     Sales|  4100|
|  Emma|     Sales|  4400|
|Olivia|     Sales|  4700|
+------+----------+------+

Group By (Wide)


In [ ]:
df_grouped = df.groupby("department").count()
df_grouped.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|department|count|
+----------+-----+
|     Sales|   10|
+----------+-----+

## Pushdown Predicate

Reduce the total data read applying conditional filters to the data columns.

- Depends on file formats (Parquet and ORC)

1. Query submitted to spark
2. Planning phase using Catalyst optimizer
3. Filters identification (predicate)
4. Pushdown: try to bring the filter to reading

Using pure spark `WHERE` clause is enough, when using _AWS Glue_, `pushdown_predicate` argument is needed.
